In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# BigQuery tables for Apache Iceberg

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-ml-recipes/main/notebooks/quickstart/iceberg/iceberg_quickstart.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/quickstart/iceberg/iceberg_quickstart.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/ai-ml-recipes/main/notebooks/quickstart/iceberg/iceberg_quickstart.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/quickstart/iceberg/iceberg_quickstart.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>

</table>

## Overview

This notebook shows how to create a BigQuery managed table for Apache Iceberg and load data from a dataset in .parquet format into this table.

#### **Steps**
Using BigQuery SQL,
1) It creates a BigQuery dataset
2) It creates a BigQuery table for Apache Iceberg with a specific schema
3) It loads data from any format to this managed table
4) It queries the table and shows the table ddl

#### **References**
- https://cloud.google.com/bigquery/docs/iceberg-tables

In [ ]:
%pip install --upgrade google-cloud-bigquery google-cloud-bigquery-connection bigframes tqdm bigquery-magics -q

In [ ]:
%load_ext bigquery_magics

In [ ]:
import bigframes.pandas as bpd
import bigframes.bigquery as bbq
bpd.options.display.progress_bar = None
import pandas as pd

In [ ]:
PROJECT_ID = "<PROJECT_ID>"
REGION = "<REGION>" 

PARQUET_FILES_GCS_URI = "gs://dataproc-metastore-public-binaries/real_estate_sales/*" # Input dataset (public)
ICEBERG_BQ_MANAGED_TABLE_GCS_URI = "gs://GCS_PATH" # Path to store the iceberg dataset

DATASET_ID = "<DATASET_ID>"
TABLE_NAME = "<TABLE_NAME>"

In [ ]:
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = REGION
bpd.options.display.progress_bar = None

Lets first take a look at the original dataset

In [ ]:
df = bpd.read_parquet(f"{PARQUET_FILES_GCS_URI[:-1]}")
df.head()

#### Create BigQuery dataset

In [ ]:
query = f"""
CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}`.{DATASET_ID} OPTIONS ( location = '{REGION}' );
"""

In [ ]:
%%bigquery --project {PROJECT_ID}
$query

#### Create a BigQuery managed Iceberg table

In [ ]:
query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}`.{DATASET_ID}.{TABLE_NAME} (
  serial_number INTEGER,
  list_year INTEGER,
  date_recorded STRING,
  town STRING,
  address STRING,
  assessed_value FLOAT64,
  sale_amount FLOAT64,
  sales_ratio FLOAT64,
  property_type STRING,
  residential_type STRING,
  non_use_code STRING,
  assessor_remarks STRING,
  opm_remarks STRING,
  longitude FLOAT64,
  latitude FLOAT64
)
CLUSTER BY town
WITH CONNECTION DEFAULT
OPTIONS (
file_format = 'PARQUET',
table_format = 'ICEBERG',
storage_uri = '{ICEBERG_BQ_MANAGED_TABLE_GCS_URI}');
"""

In [ ]:
%%bigquery --project {PROJECT_ID}
$query

#### Import data into Iceberg table

Now lets import it to this managed Iceberg table in BigQuery

In [ ]:
query = f"""
LOAD DATA INTO `{PROJECT_ID}`.{DATASET_ID}.{TABLE_NAME}
FROM FILES (
    uris=['{PARQUET_FILES_GCS_URI}'],
    format='PARQUET');
"""

In [ ]:
%%bigquery --project {PROJECT_ID}
$query

#### Query the Iceberg table

We can now query the data from this BigQuery table

In [ ]:
query = f"""
SELECT * FROM `{PROJECT_ID}`.{DATASET_ID}.{TABLE_NAME} LIMIT 5;
"""

In [ ]:
%%bigquery --project {PROJECT_ID}
$query

If we take a look at the BigQuery Information Schema, we can see the Data Definition Language for this table

In [ ]:
query = f"""
SELECT
  ddl
FROM
  {DATASET_ID}.INFORMATION_SCHEMA.TABLES;
"""

In [ ]:
%%bigquery ddl --project {PROJECT_ID} 
$query

In [ ]:
print(ddl.values[0][0])